<a href="https://colab.research.google.com/github/joris-vaneyghen/mss-jazz-playalong/blob/main/explore_audio_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# TEST segment audio

In [2]:
!git clone https://github.com/joris-vaneyghen/mss-jazz-playalong.git

fatal: destination path 'mss-jazz-playalong' already exists and is not an empty directory.
